WRITE YOUR STRATEGY IN THE CELL BELOW. THE DEFAULT STRATEGY PLAYS THE SAME AS "RANDOM" BELOW. ONCE YOU HAVE ADDED YOUR STRATEGY PRESS "SHIFT+ENTER" TO UPDATE THE CELL 

In [29]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import random as rd
import itertools
import time as t

def ScoreTeam(SuperSecretCodes='random'):
    # SuperSecretCodes[0] = code for ScoreRunner, SuperSecretCodes[1] = code for ScorePaceMaker
    if SuperSecretCodes == 'random':
        SuperSecretCodes = [list(rd.choice(list(itertools.product(['c','d'],repeat=10)))), list(rd.choice(list(itertools.product(['c','d'],repeat=10))))]
        print("SuperSecretCodes: ",SuperSecretCodes)
    
    RunnerMoves, PaceMakerMoves = SuperSecretCodes[0] + 'd'*(200 - len(SuperSecretCodes[0])), SuperSecretCodes[1] + 'c'*(200 - len(SuperSecretCodes[1]))
    # In case runner goes up against runner
    RRMoves = SuperSecretCodes[0] + 'c'*(200 - len(SuperSecretCodes[0]))

    def kindness(round):
        return 1 - (0.8)*(199-round)

    def ScoreRunner(round,P1State,P2State):
        # Is the opponent a ScorePaceMaker?
        if P2State[:round] == PaceMakerMoves[:round]:
            P1State[round] = RunnerMoves[round]
            return P1State
        
        # Is the opponent a ScoreRunner?
        if P2State[:round] == RRMoves[:round]:
            P1State[round] = RRMoves[round]
            return P1State
        
        # Otherwise, play softMAJ -- Placeholder strategy for now
        if 2*(P2State[:round].count("d")) <= round:
            P1State[round]="c"
        else:
            P1State[round]="d"
        
        # Do I want to be kind?
        if P1State[round] == "c" and rd.random() > kindness(round):
            P1State[round] = "d"

        return P1State

    def ScorePaceMaker(round,P1State,P2State):
        # Is the opponent a ScoreRunner?
        if P2State[:round] == RunnerMoves[:round]:
            P1State[round] = PaceMakerMoves[round]
            return P1State
        
        # Otherwise, play ALLD
        P1State[round]="d"
        return P1State
    
    return ScoreRunner,ScorePaceMaker


ST1 = ScoreTeam(['d', 'd', 'c'])
ST1 = [ST1[0]] + [ST1[1]]*7

ST2 = ScoreTeam(['d', 'd', 'c', 'c'])
ST2 = [ST2[0]] + [ST2[1]]*7

ST3 = ScoreTeam(['c', 'd', 'd', 'c', 'c'])
ST3 = [ST3[0]] + [ST3[1]]*7


def extort(round,P1State,P2State):
    if round == 0 : 
        P1State[round]="c"
    elif P1State[round-1] == "c" and P2State[round-1] == "c":
        if rd.random() < 8/9:
            P1State[round]="c"
        else:
            P1State[round]="d"
    elif P1State[round-1] == "c" and P2State[round-1] == "d":
        if rd.random() < 1/2:
            P1State[round]="c"
        else:
            P1State[round]="d"
    elif P1State[round-1] == "d" and P2State[round-1] == "c":
        if rd.random() < 1/3:
            P1State[round]="c"
        else:
            P1State[round]="d"
    elif P1State[round-1] == "d" and P2State[round-1] == "d":
            P1State[round]="d"
    return P1State


def guesser1(P1State,P2State):
    history = list(zip(P1State,P2State))
    c_history = [history[index-1] for index, i in enumerate(history) if i[1] == 'c']
    p_cc = c_history.count(('c','c')) / history.count(('c','c')) if history.count(('c','c')) != 0 else 1
    p_cd = c_history.count(('c','d')) / history.count(('c','d')) if history.count(('c','d')) != 0 else 1
    p_dc = c_history.count(('d','c')) / history.count(('d','c')) if history.count(('d','c')) != 0 else 1
    p_dd = c_history.count(('d','d')) / history.count(('d','d')) if history.count(('d','d')) != 0 else 1/2
    return p_cc,p_cd,p_dc,p_dd

def confidence1(P1State,P2State):
    history = list(zip(P1State,P2State))
    possible1history = [('c', 'c'), ('c', 'd'), ('d', 'c'), ('d', 'd')]
    confidence = 0
    for prev in possible1history:
        prev_history = [i[1] == 'c' for index, i in enumerate(history) if history[index-1] == prev]
        if len(prev_history) != 0:
            confidence += 2 * len([prev_history[index] != prev_history[index-1] for index, i in enumerate(prev_history) if index != 0]) / len(prev_history)
        else:
            confidence += 0.5
    return confidence/4

def simulate(p, x, startRound, endRound, initial_state=np.array([1/2, 1/2, 0/4, 0/4]).T):
    M = np.array([[p[i]*x[i], (p[i])*(1-x[i]), (1-p[i])*(x[i]), (1-p[i])*(1-x[i])] for i in range(4)]).T
    score = 0
    for _ in range(min(50, endRound-startRound)):
        initial_state = M @ initial_state
        score += np.array([1, -1, 2, 0]) @ initial_state
    return score

# BestResponse = np.loadtxt("BestResponse8.1.txt")
# print(BestResponse[1])

def Guesser1(round,P1State,P2State):
    if round < 30 : 
        P1State[round] = RATIO(round,P1State,P2State)[round]
        return P1State

    p_cc,p_cd,p_dc,p_dd = guesser1(P1State,P2State)
    p = np.array([p_cc, p_cd, p_dc, p_dd])

    previous_state = np.array([0, 0, 0, 0])
    previous_state[2*(P1State[round-1] == 'd') + (P2State[round-1] == 'd')] = 1
    
    n = 2
    X = np.array(np.meshgrid(np.linspace(0,1,n), np.linspace(0,1,n), np.linspace(0,1,n), np.linspace(0,1,n))).T.reshape(-1,4)
    scores = np.array([simulate(x, p, round+1, 200, initial_state=previous_state.T) for x in X[::-1]])
    best_x = X[::-1][np.argmax(scores)]
    # print(best_x, scores[np.argmax(scores)])
    # best_x = BestResponse[np.argmin(np.linalg.norm(BestResponse[:,:4] - p, axis=1))][4:]

    if rd.random() < best_x@previous_state:
        P1State[round] = 'c'
    else:
        P1State[round] = 'd'
    return P1State
    
def RATIO(round,P1State,P2State):
    if round == 0:
        P1State[round]="c"
    else:
        nd = P2State.count("d") # no. of defects
        nc = P2State.count("c") # no. of cooperations
        if nd == 0:
            P1State[round]="c"
        else:
            r = nc/nd
            if r > 0.925:
                P1State[round]="c";
            else:
                P1State[round]="d";
    return P1State

def RATIOk7(round,P1State,P2State):
    if round == 0:
        P1State[round]="c"
    else:
        nd = P2State.count("d") # no. of defects
        nc = P2State.count("c") # no. of cooperations
        if nd == 0:
            P1State[round]="c"
        else:
            r = nc/nd
            if r > 0.925:
                P1State[round]="c";
            else:
                P1State[round]="d";
    
    if P1State[round] == "c" and rd.random() > 1 - (0.7)**(199-round):
        P1State[round] = "d"
    return P1State

def ResponseDecider(round,P1State,P2State):
    response = [0]
    period = 8
    for _ in range(round//period):
        reaction, P2State = P2State[:period], P2State[period:]
        nd = reaction.count("d") # no. of defects
        if nd <= 1:
            response.append(-1)
        elif nd <= 4:
            response.append(0)
        else:
            response.append(1)
    return response[1:]

def TieredResponse(nthround,P1State,P2State):
    strats = [ALLD, extort, RATIO, ALLC]
    stratIndex = 1
    decisions = ResponseDecider(nthround,P1State,P2State)
    for i in decisions:
        if stratIndex == 0 and i == 1:
            stratIndex = 1
            strats = strats[1:]
        else:
            stratIndex += i
            
        if stratIndex < 0:
            stratIndex = 0
        elif stratIndex == len(strats):
            P1State = ALLD(nthround,P1State,P2State)
            return P1State
        
        # if round == 199:
        #     print(stratIndex)

    if P2State[:10] == ['d'] * 10:
        P1State = ALLD(nthround,P1State,P2State)
        return P1State

    P1State = strats[round(stratIndex)](nthround,P1State,P2State)
    return P1State
    

def HistroyIsWrittenByTheVictor(nthround, P1State, P2State):
    if nthround < 10:
        P1State = RATIO(nthround,P1State,P2State)
        return P1State
    
    p_cc,p_cd,p_dc,p_dd = guesser1(P1State[nthround//2+2:],P2State[nthround//2+2:])
    p = np.array([p_cc, p_cd, p_dc, p_dd])

    previous_state = np.array([0, 0, 0, 0])
    previous_state[2*(P1State[nthround-1] == 'd') + (P2State[nthround-1] == 'd')] = 1
    
    n = 2
    X = np.array(np.meshgrid(np.linspace(0,1,n), np.linspace(0,1,n), np.linspace(0,1,n), np.linspace(0,1,n))).T.reshape(-1,4)
    scores = np.array([simulate(x, p, nthround+1, 200, initial_state=previous_state.T) for x in X[::-1]])
    best_x = X[::-1][np.argmax(scores)]
    # print(best_x, scores[np.argmax(scores)])
    # best_x = BestResponse[np.argmin(np.linalg.norm(BestResponse[:,:4] - p, axis=1))][4:]

    if rd.random() < best_x@previous_state:
        P1State[nthround] = 'c'
    else:
        P1State[nthround] = 'd'

    P1State[:nthround//2 +2] = ['c'] * (nthround//2 +2)
    return P1State

THE FUNCTIONS BELOW ARE THE DEFAULT STRATEGIES. YOU DO NOT NEED TO EDIT THESE, BUT YOU WILL NEED TO RUN THE CELL (SHIFT+ENTER) BEFORE YOU FIRST RUN THE TOURNAMENT

In [40]:


def cTFT(round,P1State,P2State):
    if round==0:
        P1State[round]="c"
    else:
        P1State[round]=P2State[round-1]
        
    return P1State

def cTFTs(round,P1State,P2State):
    if P2State[round] == 'd':
        P1State[round] = 'd'
    elif round==0:
        P1State[round]="c"
    else:
        P1State[round]=P2State[round-1]
        
    return P1State

def cTFTd(round,P1State,P2State):
    if round==0:
        P1State[round]="c"
    elif round == 199:
        P1State[round]="d"
    else:
        P1State[round]=P2State[round-1]
        
    return P1State

def cTFTdd(round,P1State,P2State):
    if round==0:
        P1State[round]="c"
    elif round >= 200 - 2:
        P1State[round]="d"
    else:
        P1State[round]=P2State[round-1]
        
    return P1State

def cTFTddd(round,P1State,P2State):
    if round==0:
        P1State[round]="c"
    elif round >= 200 - 3:
        P1State[round]="d"
    else:
        P1State[round]=P2State[round-1]
        
    return P1State

def cTFTdddd(round,P1State,P2State):
    if round==0:
        P1State[round]="c"
    elif round >= 200 - 4:
        P1State[round]="d"
    else:
        P1State[round]=P2State[round-1]
        
    return P1State


def dTFT(round,P1State,P2State):
    
    if round==0:
        P1State[round]="d"
    else:
        P1State[round]=P2State[round-1];
        
    return P1State


def TF2T(round,P1State,P2State):
    
    if round<2:
        P1State[round]="c"
    elif P2State[round-1]=="d" and P2State[round-2]=="d":
        P1State[round]="d";
    else:
        P1State[round]="c";
        
    return P1State


def T2FT(round,P1State,P2State):
    
    if round<2:
        P1State[round]="c"
    elif P2State[round-1]=="d" or P2State[round-2]=="d":
        P1State[round]="d";
    else:
        P1State[round]="c";
        
    return P1State


def GRIM(round,P1State,P2State):
    
    if round==0:
        P1State[round]="c"
    elif P1State[round-1]=="d" or P2State[round-1]=="d":
        P1State[round]="d";
    else:
        P1State[round]="c";
        
    return P1State


def GRADUAL(round,P1State,P2State):
    
    indicesD = [i for i, x in enumerate(P2State) if x == "d"]
    nDef2=len(indicesD)
    
    if round==0:
        P1State[round]="c"
    else:
        
        nDef1=0
        for j in range(nDef2):
            if P1State[round-j-1]=="d":
                nDef1=nDef1+1
                
        if nDef2>nDef1:
            P1State[round]="d"
        else:
            P1State[round]="c";            
        
    return P1State


def softMAJ(round,P1State,P2State):
    
    indicesD = [i for i, x in enumerate(P2State) if x == "d"]
    nDef2=len(indicesD)
    
    if round==0:
        P1State[round]="c"
    elif 2*nDef2<=round:
        P1State[round]="c"
    else:
        P1State[round]="d"        
        
    return P1State

def softMAJ1(round,P1State,P2State):
    if 2*(P2State[:round].count("d"))<=round:
        P1State[round]="c"
    else:
        P1State[round]="d"         
    return P1State

def softMAJs(round,P1State,P2State):
    if 2*(P2State[:round].count("d"))<=round and P2State[round] != "d":
        P1State[round]="c"
    else:
        P1State[round]="d"         
    return P1State

def hardMAJ(round,P1State,P2State):
    
    indicesD = [i for i, x in enumerate(P2State) if x == "d"]
    nDef2=len(indicesD)
    
    if round==0:
        P1State[round]="d"
    elif 2*nDef2<round:
        P1State[round]="c";
    else:
        P1State[round]="d";           
        
    return P1State



def PROBER(round,P1State,P2State):
   
    if round==0:
        P1State[round]="d"
    elif round==1:
        P1State[round]="c";
    elif round==2:
        P1State[round]="c"; 
    elif P2State[1]=="c" and P2State[2]=="c":
        P1State[round]="d";
    else:
        P1State[round]=P2State[round-1];
        
    return P1State



def GTFT(round,P1State,P2State):
    cost = 1
    benefit = 2
    
    if round==0:
        P1State[round]="c"
    elif P1State[round-1]=="c" and P2State[round-1]=="c":
        P1State[round]="c"
    elif P1State[round-1]=="d" and P2State[round-1]=="c":
        P1State[round]="c"
    elif P1State[round-1]=="c" and P2State[round-1]=="d":
        if random.rand()<1-cost/benefit:
            P1State[round]="c"
        else:
            P1State[round]="d"
    elif P1State[round-1]=="d" and P2State[round-1]=="d":
        if random.rand()<1-cost/benefit:
            P1State[round]="c"
        else:
            P1State[round]="d"
        
    return P1State

    
    
def dWSLS(round,P1State,P2State):
    
    if round==0:
        P1State[round]="d"
    elif P1State[round-1]=="c" and P2State[round-1]=="c":
        P1State[round]="c"
    elif P1State[round-1]=="d" and P2State[round-1]=="c":
        P1State[round]="d"
    elif P1State[round-1]=="c" and P2State[round-1]=="d":
        P1State[round]="d"
    elif P1State[round-1]=="d" and P2State[round-1]=="d":
        P1State[round]="c"
        
    return P1State


def cWSLS(round,P1State,P2State):
    
    if round==0:
        P1State[round]="c"
    elif P1State[round-1]=="c" and P2State[round-1]=="c":
        P1State[round]="c"
    elif P1State[round-1]=="d" and P2State[round-1]=="c":
        P1State[round]="d"
    elif P1State[round-1]=="c" and P2State[round-1]=="d":
        P1State[round]="d"
    elif P1State[round-1]=="d" and P2State[round-1]=="d":
        P1State[round]="c"
        
    return P1State
    
    
def ALLD(round,P1State,P2State):
    P1State[round]="d";
    return P1State

def ALLC(round,P1State,P2State):
    P1State[round]="c";
    return P1State


def RANDOM(round,P1State,P2State):
    if random.rand()<0.5:
        P1State[round]="c";
    else:
        P1State[round]="d";
    return P1State

def cTFTsCopy(round,P1State,P2State):
    if P2State[round] == None:
        return cTFT(round,P1State,P2State)
    else:
        P1State[round] = P2State[round]
        return P1State

def softMAJsCopy(round,P1State,P2State):
    if P2State[round] == None:
        return softMAJs(round,P1State,P2State)
    else:
        P1State[round] = P2State[round]
        return P1State

def cTFTsCopydd(round,P1State,P2State):
    if P2State[round] == None:
        return cTFTdd(round,P1State,P2State)
    else:
        if round >= 200 - 2:
            P1State[round] = "d"
        else:
            P1State[round] = P2State[round]
        return P1State
    
def softMAJsCopydd(round,P1State,P2State):
    if P2State[round] == None:
        return softMAJs(round,P1State,P2State)
    else:
        if round >= 200 - 2:
            P1State[round] = "d"
        else:
            P1State[round] = P2State[round]
        return P1State

#THIS FUNCTION RUNS 200 ROUNDS OF THE GAME BETWEEN TWO STRATEGIES

# def Game(Strat1,Strat2,LRounds, verbose=False) -> tuple[float,float,int,int]:
#     # benefit = 2
#     # cost = 1

#     # Payoff1=0
#     # Payoff2=0
#     P1State=[None]*LRounds
#     P2State=[None]*LRounds

#     for i in range(LRounds):
#         Strat1(i,P1State,P2State)
#         Strat2(i,P2State,P1State)
        
#         # if P1State[i]=="c":
#         #     Payoff1-=cost
#         #     Payoff2+=benefit
            
#         # if P2State[i]=="c":
#         #     Payoff2-=cost
#         #     Payoff1+=benefit

#     if verbose:
#         print('P1State : ', P1State)
#         print('P2State : ', P2State)

#     Payoff1=-P1State.count("c")+2*P2State.count("c")
#     Payoff2=-P2State.count("c")+2*P1State.count("c")
#     #print(P1State,P2State)    

#     Win1 = Payoff1>Payoff2
#     Win2 = Payoff2>Payoff1

#     # return Payoff1/LRounds,Payoff2/LRounds
#     return Payoff1/LRounds,Payoff2/LRounds,Win1,Win2

def Game(Strat1,Strat2,LRounds, verbose=False) -> tuple[float,float,int,int]:
    benefit = 2
    cost = 1
    P1State=[None]*LRounds
    P2State=[None]*LRounds
    vP1State=[None]*LRounds
    Payoff1=0
    Payoff2=0

    TrueRecordP1, TrueRecordP2 = [], []

    for i in range(LRounds):
        Strat1(i,P1State,P2State)
        
        if i>0:
            vP1State[0:i]=P1State[0:i]           
        
        Strat2(i,P2State,vP1State)
        
        if P1State[i]=="c":
            Payoff1=Payoff1-cost
            Payoff2=Payoff2+benefit
            
        if P2State[i]=="c":
            Payoff2=Payoff2-cost
            Payoff1=Payoff1+benefit
            
        TrueRecordP1.append(P1State[i])
        TrueRecordP2.append(P2State[i])

    if Payoff1>Payoff2:
        Win1=1
        Win2=0
    elif Payoff2>Payoff1:
        Win1=0
        Win2=1
    else:
        Win1=0
        Win2=0
    
    if verbose:
        print('P1State : ', TrueRecordP1)
        print('P2State : ', TrueRecordP2)

    #print(P1State,P2State,vP1State)  
    return Payoff1/LRounds,Payoff2/LRounds,Win1,Win2

def ScoreTeamVersitile(SuperSecretShortCode='r', defaultStrat=softMAJ):
    if SuperSecretShortCode == 'r':
        SuperSecretShortCode = list(rd.choice(list(itertools.product(['c','d'],repeat=5))))
        print("SuperSecretShortCode: ",SuperSecretShortCode)

    def ScoreRunner(round,P1State,P2State):
        if round < len(SuperSecretShortCode):
            P1State[round]=SuperSecretShortCode[round]
            return P1State
        
        if P2State[:len(SuperSecretShortCode)] == SuperSecretShortCode:
            P1State[round]="d"
            return P1State
        
        round1 = round - len(SuperSecretShortCode)
        P1State1 = P1State[len(SuperSecretShortCode):].copy()
        P2State1 = P2State[len(SuperSecretShortCode):].copy()
        P1State[round] = defaultStrat(round1,P1State1,P2State1)[round1]
        return P1State

    def ScorePaceMaker(round,P1State,P2State):
        if round < len(SuperSecretShortCode):
            P1State[round]=SuperSecretShortCode[round]
            return P1State
        
        if P2State[:2*len(SuperSecretShortCode)] == SuperSecretShortCode + ['d']*len(SuperSecretShortCode) and 2*len(SuperSecretShortCode) < round:
            P1State[round]="c"
            return P1State
        
        P1State[round]="d"
        return P1State
    return ScoreRunner,ScorePaceMaker

def ScoreTeamVersitileList(SuperSecretShortCode, defaultStrat=softMAJ):
    return [ScoreTeamVersitile(SuperSecretShortCode, defaultStrat)[0]] + [ScoreTeamVersitile(SuperSecretShortCode, defaultStrat)[1]]*7

In [ ]:
memory=1
fidelity = 2

def manifest1(strat_gene : dict):
    def gene_carrier(round, P1State, P2State):
        # print("P1State",P1State)
        if round == 0 :
            P1State[round] = 'c' if strat_gene.get('', 0) > random.random() else 'd'
        else:
            P1State[round] = 'c' if strat_gene.get((P1State[round-1]+ P2State[round-1]), None) > random.random() else 'd'
        return P1State
    return gene_carrier

possible_history = [''.join(x) for i in range(memory+1) for x in itertools.product(['c','d'], repeat=i*2)]
A = list(itertools.product(*([np.linspace(0,1,fidelity)]*len(possible_history))))
stratSpace = [manifest1({possible_history[i]:float(a[i]) for i in range(len(possible_history))}) for a in A]

# show me the results of Game(stratSpace[i], stratSpace[j], 200) for all i,j
# It should be formatted as a matrix

def score_matrix(stratSpace):
    result = np.zeros((len(stratSpace),len(stratSpace)))
    for i in range(len(stratSpace)):
        for j in range(len(stratSpace)):
            result[i,j] = Game(stratSpace[i], stratSpace[j], 200)[0]
    return result

with np.printoptions(threshold=np.inf):
    print(score_matrix(stratSpace))


THE TOURNAMENT IS RUN IN THE CELL BELOW. ADD THE NAME OF YOUR STRATEGY/STRATEGIES TO "StratList", THEN RUN THE CODE. TOURNAMENT RESULTS ARE DISPLAYED BELOW. IF YOU WANT TO RUN A TOURNAMENT WITH ONLY A SUBSET OF THE STRATEGIES, YOU CAN JUST EDIT "StratList" TO INCLUDE ONLY THE STRATEGIES YOU ARE INTERESTED IN

In [ ]:
def interleave(P1State,P2State):
    return tuple([x for t in zip(P1State,P2State) for x in t if x != None][-2*1:])

interleave(["c","d","c"],["d","c","d"])
# interleave([], [])


In [ ]:
# Code the genetic algorithm here
pop = 50
gen = 200
Rounds = 200
# benefit=2
# cost=1
memory=2
replacementrate=0.5
fidelity = 1


def manifest1(strat_gene : dict):
    def gene_carrier(round, P1State, P2State):
        # print("P1State",P1State)
        if round == 0 :
            P1State[round] = 'c' if strat_gene.get('', 0) > random.random() else 'd'
        else:
            P1State[round] = 'c' if strat_gene.get((P1State[round-1]+ P2State[round-1]), None) > random.random() else 'd'
        return P1State
    return gene_carrier

def manifest2(strat_gene : dict):
    def gene_carrier(round, P1State, P2State):
        # print("P1State",P1State)
        if round == 0 :
            P1State[round] = 'c' if strat_gene.get('', 0) > random.random() else 'd'
        elif round == 1:
            P1State[round] = 'c' if strat_gene.get(P1State[round-1]+ P2State[round-1], None) > random.random() else 'd'
        else:
            P1State[round] = 'c' if strat_gene.get(P1State[round-2]+ P2State[round-2]+ P1State[round-1]+ P2State[round-1], None) > random.random() else 'd'
        return P1State
    return gene_carrier

# Create a population of random strategies
# Each strategy is a dictionary with the game history so far as the key and the probability of playing C as the value
# keys are tuples length n with each element of the form (P1,P2) where P1 is the last move of player 1 and P2 is the last move of player 2
possible_history = [''.join(x) for i in range(memory+1) for x in itertools.product(['c','d'], repeat=i*2)]
mutationrate=3/len(possible_history)


population = [{history : round(random.random(), fidelity) for history in possible_history} for _ in range(pop)]
# for i in range(pop):
#     strat = {}
#     for history in possible_history:
#         strat[history] = round(random.random(), 5)
#     population.append(strat)

# Initial doping of the population
# one for me, one for you
population[10] = {'': 1, 'cc': 1, 'cd': 1, 'dc': 0, 'dd': 0, 'cccc': 1, 'cccd': 0.5, 'ccdc': 0.7, 'ccdd': 0.8, 'cdcc': 1, 'cdcd': 0, 'cddc': 0.6, 'cddd': 0, 
                   'dccc': 0.7, 'dccd': 0.6, 'dcdc': 1, 'dcdd': 1, 'ddcc': 0, 'ddcd': 0.7, 'dddc': 1, 'dddd': 0}
# population[25] = {'': 0.27799, 'cc': 0.2203, 'cd': 2, 'dc': 0.75391, 'dd': 0.36862, 'cccc': 0.31446, 'cccd': 0.29074, 'ccdc': 0.50654, 'ccdd': 0.26469, 'cdcc': 0.30972, 'cdcd': 0.32928, 'cddc': 2, 'cddd': -1, 'dccc': -1, 'dccd': -1, 'dcdc': 0.1271, 'dcdd': -1, 'ddcc': 0.29599, 'ddcd': -1, 'dddc': 0.66114, 'dddd': 0.24065}
# population[30] = {'': 1, 'cc': 1, 'cd': 0, 'dc': 1, 'dd': 0, 'cccc': 1, 'cccd': 0, 'ccdc': 0.50654, 'ccdd': 0, 'cdcc': 1, 'cdcd': 0, 'cddc': 2, 'cddd': -1, 'dccc': -1, 'dccd': -1, 'dcdc': 0, 'dcdd': -1, 'ddcc': 1, 'ddcd': -1, 'dddc': 0, 'dddd': 0}
# population[0] = {i:0 for i in possible_history}
# for i in range(1, 2):
#     population[i][('dddd')] = 0
#     population[i][('cddd')] = 0
#     population[i][('cdcd')] = 0
#     population[i][('')] = 1
#     population[i][('cc')] = 1
#     population[i][('cccc')] = 1

# Run the genetic algorithm
for g in range(gen):
    timestart = t.time()
    populationExtended = list(map(manifest2, population)) + [ALLD]*pop + ST1*3 + ST2 + ST3
    # Calculate the fitness of each strategy
    fitness = [0 for _ in range(len(populationExtended))]
    for i in range(len(populationExtended)):
        for j in range(i+1,len(populationExtended)):
            Result=Game(populationExtended[i],populationExtended[j],Rounds)
            # print(Result)
            fitness[i]+=Result[0]
            fitness[j]+=Result[1]
    # print(fitness)
    # Sort the population by fitness
    population = [x for _,x in sorted(zip(fitness[:pop],population), key=lambda pair: pair[0])]
    fitness = sorted(fitness[:pop])
    best_fitness = fitness[-1]
    best_individual = population[-1]
    # Normalise the fitness : some fitnesses may be negative so we add the minimum fitness to all fitnesses to make them all positive
    fitness = [x-min(fitness) for x in fitness]
    # if sum(fitness) == 0:
    #     print("All fitnesses are 0")
    #     print(population)
    #     print(fitness)
    #     break
    fitness = [x/sum(fitness) for x in fitness]
    # Remove the least fit strategies
    newpopulation = population[int(replacementrate*pop):]
    # Create new strategies by mutation and crossover
    for i in range(int(replacementrate*pop)):
        # Select two random strategies from the population
        strat1 = random.choice(population, p=fitness)
        strat2 = random.choice(population, p=fitness)
        # Create a new strategy by crossover
        new_strat = strat1.copy()
        # pick a random history from possible_history
        cutpoint = random.choice(possible_history)
        for history in possible_history:
            if history[:len(cutpoint)] == cutpoint:
                new_strat[history] = strat2[history]
        # cutpoint = random.randint(0,len(possible_history))
        # for index, history in enumerate(possible_history):
        #     if index < cutpoint:
        #         new_strat[history] = strat1[history]
        #     else:
        #         new_strat[history] = strat2[history]
        # Mutate the new strategy
        for history in possible_history:
            if random.random() < mutationrate:
                new_strat[history] = round(new_strat[history] + (random.random()- 0.5)/3, fidelity) 
        newpopulation.append(new_strat)
    population = newpopulation
    print(f"Gen {g}| T", round(t.time()-timestart, 3), "| best fitness:", '%.1f' % best_fitness, '| best individual:',  best_individual)


In [ ]:
# champ = manifest2({'': 1, 'cc': 1, 'cd': 0.2, 'dc': 0.8, 'dd': 1, 'cccc': 0.3, 'cccd': 0, 'ccdc': 0.3, 'ccdd': 1, 'cdcc': 0, 'cdcd': 0.3, 'cddc': 1, 'cddd': 0, 
#                     'dccc': 0.3, 'dccd': 0, 'dcdc': 1, 'dcdd': 1, 'ddcc': 0, 'ddcd': 0, 'dddc': 1, 'dddd': 0})
champ = manifest2({'': 1, 'cc': 1, 'cd': 1, 'dc': 0, 'dd': 0, 'cccc': 1, 'cccd': 0.5, 'ccdc': 0.7, 'ccdd': 0.8, 'cdcc': 1, 'cdcd': 0, 'cddc': 0.6, 'cddd': 0, 
                   'dccc': 0.7, 'dccd': 0.6, 'dcdc': 1, 'dcdd': 1, 'ddcc': 0, 'ddcd': 0.7, 'dddc': 1, 'dddd': 0})

In [41]:
# Arena for 1 v 1 detailed analysis

# gene1 = {i:1 for i in possible_history}
# gene1[('dddd')] = 0
# gene1[('cddd')] = 0
# gene1[('cdcd')] = 0

# gene2 = {i:1 for i in possible_history}
# gene2[('dddd')] = 0
# gene2[('cddd')] = 0
# gene2[('cdcd')] = 0
# gene2 = {i:0 for i in possible_history}

# gene1 = {'': 1, 'cc': 1, 'cd': 0.24551, 'dc': 0.45215, 'dd': 0.37644, 'cccc': 1, 'cccd': 0.00104, 'ccdc': 0.85526, 'ccdd': 0.41678, 'cdcc': 0.1638, 'cdcd': 0, 'cddc': 0.73728, 'cddd': 0, 'dccc': 0.78639, 'dccd': 0.22014, 'dcdc': 0.94384, 'dcdd': 0.5982, 'ddcc': 0.01756, 'ddcd': 0.41327, 'dddc': 0.24606, 'dddd': 0}
# gene2 = {'': 1, 'cc': 1, 'cd': 0.09939, 'dc': 0.06266, 'dd': 0.84993, 'cccc': 1, 'cccd': 0.72082, 'ccdc': 0.52644, 'ccdd': 0.7422, 'cdcc': 0.96867, 'cdcd': 0, 'cddc': 0.61291, 'cddd': 0, 'dccc': 0.3761, 'dccd': 0.56994, 'dcdc': 0.77985, 'dcdd': 0.16429, 'ddcc': 0.59043, 'ddcd': 0.95448, 'dddc': 0.03142, 'dddd': 0}

# p1 = manifest2(gene1)
# p2 = manifest2(gene2)

# Game(p1,p2,200, verbose=True)

# for strat in [cTFT,dTFT,cWSLS,dWSLS,ALLD,ALLC,RANDOM,GTFT,TF2T,T2FT,GRIM,GRADUAL,softMAJ,hardMAJ,PROBER]:
#     if not Game(WinRunner, strat, 200)[2]:
#         print("WinRunner don't win against to", strat.__name__)
#         print(Game(WinRunner, strat, 200, verbose=True))

# print()

# for strat in [cTFT,dTFT,cWSLS,dWSLS,ALLD,ALLC,RANDOM,GTFT,TF2T,T2FT,GRIM,GRADUAL,softMAJ,hardMAJ,PROBER]:
#     if not Game(ALLD, strat, 200)[2]:
#         print("ALLD don't win against to", strat.__name__)
#         print(Game(ALLD, strat, 200, verbose=True))

# Game(cTFT, ST1[0], 200, verbose=True)
# Game(ST1[0], cTFT, 200, verbose=True)

# Game(SP_softMAJ2, SP_cTFT1, 200, verbose=True)


# n = 2
# X = np.array(np.meshgrid(np.linspace(0,1,n), np.linspace(0,1,n), np.linspace(0,1,n), np.linspace(0,1,n))).T.reshape(-1,4)
# scores = np.array([simulate([1, 1, 0, 0], x, 10+1, 200, initial_state=[0, 0, 1, 0]) for x in X])
# best_x = X[np.argmax(scores)]
# for x, score in zip(X, scores):
#     print(x, score)

# def simulate(p, x, remaining_rounds, initial_state=np.array([1/4, 1/4, 1/4, 1/4]).T):
#     M = np.array([[p[i]*x[i], (1-p[i])*x[i], p[i]*(1-x[i]), (1-p[i])*(1-x[i])] for i in range(4)]).T
#     # S = np.sum([M**i for i in range(remaining_rounds+1)], axis=0)
#     # A = (S @ initial_state)
#     # score = np.array([1, -1, 2, 0]) @ A
#     score = 0
#     for _ in range(remaining_rounds):
#         initial_state = M @ initial_state
#         score += np.array([1, -1, 2, 0]) @ initial_state
#     return score

# print(simulate([1, 1, 0, 0], [0, 0, 1, 1], 0, 100, initial_state=[0, 0, 1, 0]))

for stratOpponent in [cTFT,dTFT,cWSLS,dWSLS,ALLD,ALLC,RANDOM,GTFT,TF2T,T2FT,GRIM,GRADUAL,softMAJ,hardMAJ,PROBER]:
    print(f'stratOpponent is {stratOpponent} :')
    print(Game(RATIO, stratOpponent, 200, verbose=True))
    print(Game(TieredResponse, stratOpponent, 200, verbose=True))
    print(Game(HistroyIsWrittenByTheVictor, stratOpponent, 200, verbose=True))
    print(Game(extort, stratOpponent, 200, verbose=True))

stratOpponent is <function cTFT at 0x000001C0B03F25F0> :
P1State :  ['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', '

In [33]:
# STd = ScoreTeamVersitileList(['d', 'd', 'c', 'c'], cTFTsCopydd)

# StratList=[cTFT,TF2T,T2FT,softMAJ1, RATIOk7]*2 + [ALLD] * 10 + ST1 + ST2 + [cTFTd, TieredResponse]
# StratList= ST1 + [cTFTsCopydd]*10 + [cTFT,dTFT,cWSLS,dWSLS,ALLC,RANDOM,GTFT,TF2T,T2FT,GRIM,GRADUAL,softMAJ,hardMAJ,PROBER]
# rd.shuffle(StratList)
# StratList = [cTFTsCopydd] + StratList + [cTFTsCopydd] + STd
# SR_softMAJ1, SP_softMAJ1 = ScoreTeamVersitile(defaultStrat=softMAJ)
# SR_softMAJ2, SP_softMAJ2 = ScoreTeamVersitile(defaultStrat=softMAJ)
# SR_cTFT1, SP_cTFT1 = ScoreTeamVersitile(defaultStrat=cTFT)
# SR_cTFT2, SP_cTFT2 = ScoreTeamVersitile(defaultStrat=cTFT)
# StratList = [SR_softMAJ1] + [SP_softMAJ1]*7 + [SR_cTFT1] + [SP_cTFT1]*7 + [SR_softMAJ2] + [SP_softMAJ2]*7 + [ALLD] * 40

StratList = [RATIO, TieredResponse, HistroyIsWrittenByTheVictor, cTFT,dTFT,cWSLS,dWSLS,ALLC, ALLD,RANDOM,GTFT,TF2T,T2FT,GRIM,GRADUAL,softMAJ,hardMAJ,PROBER]*2

#RUN THE CODE

Rounds=200
benefit=2
cost=1

PayoffTot=[0]*len(StratList)
WinTot=[0]*len(StratList)

for i in range(len(StratList)):
    for j in range(i+1,len(StratList)):
        Result=Game(StratList[i],StratList[j],Rounds)
        PayoffTot[i]=PayoffTot[i]+Result[0]
        PayoffTot[j]=PayoffTot[j]+Result[1]
        WinTot[i]=WinTot[i]+Result[2]
        WinTot[j]=WinTot[j]+Result[3]
        #print(StratList[i].__name__,"vs",StratList[j].__name__,":",StratList[i].__name__,"payoff=",Result[0],",",StratList[j].__name__,"payoff=",Result[1])
        
ScInd=sorted(range(len(PayoffTot)),reverse=True, key=lambda k: PayoffTot[k])

print("Tournament Outcome by Score:")
print()

k=1

for i in ScInd:
        print(k,StratList[i].__name__,":","Score=",round(PayoffTot[i],3))
        k=k+1

print()
print()
        
WinInd=sorted(range(len(PayoffTot)),reverse=True, key=lambda k: WinTot[k])

print("Tournament Outcome by Wins:")
print()

k=1

for i in WinInd:
        print(k,StratList[i].__name__,":","Wins=",WinTot[i])
        k=k+1

'''
Tournament Outcome by Score:

1 ScoreRunner : Score= 125.07
2 ScoreRunner : Score= 124.47
3 cTFTd : Score= 100.375
'''

Tournament Outcome by Score:

1 HistroyIsWrittenByTheVictor : Score= 29.965
2 HistroyIsWrittenByTheVictor : Score= 29.945
3 cTFT : Score= 28.3
4 cTFT : Score= 27.945
5 GTFT : Score= 27.275
6 RATIO : Score= 26.975
7 GTFT : Score= 26.805
8 T2FT : Score= 26.56
9 T2FT : Score= 26.34
10 RATIO : Score= 25.8
11 TF2T : Score= 25.4
12 TF2T : Score= 25.325
13 softMAJ : Score= 25.095
14 softMAJ : Score= 24.145
15 GRADUAL : Score= 24.105
16 GRADUAL : Score= 24.065
17 GRIM : Score= 23.415
18 GRIM : Score= 23.365
19 TieredResponse : Score= 22.84
20 TieredResponse : Score= 22.21
21 cWSLS : Score= 20.63
22 cWSLS : Score= 20.44
23 PROBER : Score= 19.68
24 PROBER : Score= 19.295
25 dTFT : Score= 17.585
26 dTFT : Score= 17.44
27 hardMAJ : Score= 16.72
28 hardMAJ : Score= 16.71
29 RANDOM : Score= 14.08
30 ALLC : Score= 13.32
31 ALLC : Score= 13.27
32 RANDOM : Score= 13.17
33 ALLD : Score= 12.49
34 ALLD : Score= 12.14
35 dWSLS : Score= 4.81
36 dWSLS : Score= 4.725


Tournament Outcome by Wins:

1 ALLD : Wi

'\nTournament Outcome by Score:\n\n1 ScoreRunner : Score= 125.07\n2 ScoreRunner : Score= 124.47\n3 cTFTd : Score= 100.375\n'

In [ ]:
import numpy as np
import sympy as sp


p_ALLC = np.array([1, 1, 1, 1])
p_ALLD = np.array([0, 0, 0, 0])
p_extort = np.array([8/9, 1/2, 1/3, 0])
p_TFT = np.array([1, 1, 0, 0])
p_GTFT = np.array([1, 1, 1/2, 1/2])
p_RANDOM = np.array([1/2, 1/2, 1/2, 1/2])
p_ZD54 = np.array([1, 1, 0.2, 0.2])
p_ZD53 = np.array([1, 0.75, 0.4, 0.15])

p = p_ZD53
n = 6
remaining_rounds = 50
valuation = np.array([1, -1, 2, 0])

def simulate(p, x, remaining_rounds, initial_state=np.array([1/4, 1/4, 1/4, 1/4]).T):
    M = np.array([[p[i]*x[i], (1-p[i])*x[i], p[i]*(1-x[i]), (1-p[i])*(1-x[i])] for i in range(4)]).T
    score = 0
    for _ in range(remaining_rounds):
        initial_state = M @ initial_state
        score += valuation @ initial_state
    return score

X = np.array(np.meshgrid(np.linspace(0,1,n), np.linspace(0,1,n), np.linspace(0,1,n), np.linspace(0,1,n))).T.reshape(-1,4)

scoredict = {tuple(x) : simulate(p, x, remaining_rounds) for x in X}
l = list(scoredict.items())
l.sort(key=lambda x: x[1])
print(tuple(map(lambda x: "%.2f" % x, l[0][0])), f", {l[0][1]:.2f}")
print(tuple(map(lambda x: "%.2f" % x, l[len(l)//4][0])), f", {l[len(l)//4][1]:.2f}")
print(tuple(map(lambda x: "%.2f" % x, l[len(l)//2][0])), f", {l[len(l)//2][1]:.2f}")
print(tuple(map(lambda x: "%.2f" % x, l[len(l)//4*3][0])), f", {l[len(l)//4*3][1]:.2f}")
print(tuple(map(lambda x: "%.2f" % x, l[-1][0])), f", {l[-1][1]:.2f}")

# print(best_x, f'{best_score:.2f}, {avg_score:.2f}')

'''
p = p_ALLC
('1.00', '1.00', '1.00', '1.00') , 50.00
('0.60', '0.00', '1.00', '0.00') , 64.51
('1.00', '0.40', '0.00', '0.60') , 75.00
('0.00', '1.00', '0.40', '0.80') , 85.53
('0.00', '0.00', '0.00', '0.00') , 100.00

p = p_ALLD
('1.00', '1.00', '1.00', '1.00') , -50.00
('0.00', '0.60', '0.00', '1.00') , -35.49
('0.60', '0.00', '0.00', '1.00') , -25.00
('1.00', '0.00', '0.80', '0.40') , -14.47
('0.00', '0.00', '0.00', '0.00') , 0.00

p = p_extort
('0.00', '0.00', '0.00', '0.00') , 1.29
('0.40', '0.40', '0.00', '0.40') , 9.39
('0.80', '0.60', '0.40', '0.20') , 13.78
('0.60', '0.60', '0.60', '0.80') , 18.13
('1.00', '1.00', '1.00', '1.00') , 30.55

p = p_TFT
('0.00', '0.00', '0.00', '0.00') , 1.00
('0.00', '0.00', '0.20', '1.00') , 18.08
('0.60', '0.00', '1.00', '0.40') , 25.00
('1.00', '0.60', '0.80', '0.00') , 31.92
('1.00', '1.00', '1.00', '1.00') , 49.00

p = p_GTFT
('1.00', '0.60', '0.80', '0.80') , 49.50
('0.80', '0.00', '0.80', '0.60') , 49.80
('0.60', '0.40', '0.40', '0.80') , 49.97
('0.80', '0.00', '0.20', '0.20') , 50.13
('0.20', '1.00', '0.00', '0.00') , 50.50

p = p_RANDOM
('1.00', '1.00', '1.00', '1.00') , 0.00
('1.00', '0.40', '0.60', '0.40') , 18.78
('0.40', '1.00', '0.20', '0.40') , 25.00
('0.00', '1.00', '0.20', '0.40') , 31.22
('0.00', '0.00', '0.00', '0.00') , 50.00

p = p_ZD54
('0.00', '0.00', '0.00', '0.00') , 20.80
('0.80', '0.20', '0.60', '0.00') , 31.61
('0.00', '1.00', '0.80', '0.20') , 35.68
('0.80', '0.60', '0.60', '0.40') , 40.21
('1.00', '1.00', '1.00', '1.00') , 49.20

p = p_ZD53
('0.00', '0.00', '0.00', '0.00') , 21.00
('0.00', '0.60', '0.40', '0.40') , 31.45
('0.80', '0.20', '0.40', '0.40') , 35.39
('0.60', '0.40', '1.00', '0.80') , 39.87
('1.00', '1.00', '1.00', '1.00') , 48.87
'''
